<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Information" data-toc-modified-id="Information-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Information</a></span></li><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Preprocessing</a></span></li><li><span><a href="#성별-리뷰를-얼마나-남기는지" data-toc-modified-id="성별-리뷰를-얼마나-남기는지-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>성별 리뷰를 얼마나 남기는지</a></span></li><li><span><a href="#성별-연령-비율" data-toc-modified-id="성별-연령-비율-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>성별 연령 비율</a></span></li><li><span><a href="#성별-피부타입" data-toc-modified-id="성별-피부타입-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>성별 피부타입</a></span></li><li><span><a href="#성별-선호도-비율" data-toc-modified-id="성별-선호도-비율-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>성별 선호도 비율</a></span></li><li><span><a href="#성별-평점-비율" data-toc-modified-id="성별-평점-비율-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>성별 평점 비율</a></span></li><li><span><a href="#성별에-따라-카테고리-빈도" data-toc-modified-id="성별에-따라-카테고리-빈도-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>성별에 따라 카테고리 빈도</a></span></li><li><span><a href="#성별-사용-화장품의-가격-/-카테고리별-가격" data-toc-modified-id="성별-사용-화장품의-가격-/-카테고리별-가격-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>성별 사용 화장품의 가격 / 카테고리별 가격</a></span></li><li><span><a href="#성별-사용-화장품-용량" data-toc-modified-id="성별-사용-화장품-용량-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>성별 사용 화장품 용량</a></span></li><li><span><a href="#성별-리뷰를-남기는-브랜드/상품/카테고리/유형" data-toc-modified-id="성별-리뷰를-남기는-브랜드/상품/카테고리/유형-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>성별 리뷰를 남기는 브랜드/상품/카테고리/유형</a></span></li></ul></div>

In [2]:
import pandas as pd
import os 

import pyecharts as pye

print(os.listdir('../dataset'))

['glowpick_products.csv', 'glowpick_reviews.csv', 'glowpick_reviews_old.csv', 'oliveyoung_product_info.csv']


# Load Data

In [3]:
products = pd.read_csv('../dataset/glowpick_products.csv')
reviews = pd.read_csv('../dataset/glowpick_reviews.csv')
print('product shape: ',products.shape)
print('reviews shape: ',reviews.shape)

product shape:  (1183, 8)
reviews shape:  (5208, 7)


# Information

In [4]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1183 entries, 0 to 1182
Data columns (total 8 columns):
brand          1183 non-null object
product        1183 non-null object
vol_price      1183 non-null object
rate           1183 non-null float64
nb_reviews     1183 non-null object
sales_rank     1183 non-null object
product_url    1183 non-null object
category       1183 non-null object
dtypes: float64(1), object(7)
memory usage: 74.1+ KB


In [5]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5208 entries, 0 to 5207
Data columns (total 7 columns):
date             5208 non-null object
user_id          5208 non-null object
sex              5208 non-null object
age_skin_type    5208 non-null object
rate             5208 non-null object
content          5208 non-null object
product_url      5208 non-null object
dtypes: object(7)
memory usage: 284.9+ KB


In [9]:
reviews.groupby('product_url')['content'].count().sort_values()

product_url
/product/27711       1
/product/127106      1
/product/12835       1
/product/4117        1
/product/4116        1
                  ... 
/product/34094     100
/product/1412      100
/product/6892      100
/product/11071     100
/product/2324      100
Name: content, Length: 698, dtype: int64

# Preprocessing

In [28]:
products = products.rename(columns={'rate':'star'})
glowpick = pd.merge(reviews, products, on='product_url', how='left')
print('glowpick shape: ',glowpick.shape)

glowpick shape:  (5801, 14)


In [29]:
# make unique id
glowpick['unique_id'] = np.arange(glowpick.shape[0])

In [30]:
# sex type
sex_dict = {
    'f':'Female',
    'm':'Male'
}
glowpick.sex = glowpick.sex.map(sex_dict)

In [31]:
# age and skin_type
age_skin_type_df = glowpick.age_skin_type.str.replace('·','').str.split('  ', expand=True)
age_skin_type_df.columns = ['age','skin_type']
age_skin_type_df.skin_type = age_skin_type_df.skin_type.str.strip()
age_skin_type_df.age = age_skin_type_df.age.str.replace('세','').astype(int)
glowpick = pd.concat([glowpick,age_skin_type_df], axis=1)

remove_id = glowpick[glowpick.age > 100].unique_id.tolist()
remove_id.extend(glowpick[glowpick.age < 10].unique_id.tolist())
print('이상치 수: ',len(remove_id))
glowpick = glowpick[~glowpick.unique_id.isin(remove_id)]

def age_category(x):
    for i in range(10,110,10):
        if x < i:
            return f'{i}대'

glowpick['age'] = glowpick.age.map(age_category)

이상치 수:  22


# 성별 리뷰를 얼마나 남기는지

In [32]:
sex_cnt = glowpick.sex.value_counts()
pie_sex = pye.Pie('성별 리뷰 비율',title_pos='center',width=400, height=300)
pie_sex.add(' ',sex_cnt.index, sex_cnt,
            radius=[20, 60],
            is_label_show=True,
            legend_orient='vertical',
            legend_pos='left')
pie_sex

In [33]:
pie_sex.render('../images/graphs/sex_nbreviews.html')

# 성별 연령 비율

In [34]:
f_age_cnt = glowpick.loc[glowpick.sex=='Female','age'].value_counts()
m_age_cnt = glowpick.loc[glowpick.sex=='Male','age'].value_counts()

pie_age = pye.Pie('성별 연령대 비율 [좌: 여성, 우: 남성]',title_pos='center')
pie_age.add('여성',f_age_cnt.index, f_age_cnt,
            center=[30,50],
            radius=[20,50],
           is_label_show=True)
pie_age.add('남성',m_age_cnt.index, m_age_cnt,
            center=[70,50],
            radius=[20,50],
           is_label_show=True,
           legend_orient='vertical',
           legend_pos='left')
pie_age

In [35]:
pie_age.render('../images/graphs/sex_age.html')

# 성별 피부타입

In [36]:
f_skin_type_cnt = glowpick.loc[glowpick.sex=='Female','skin_type'].value_counts()
m_skin_type_cnt = glowpick.loc[glowpick.sex=='Male','skin_type'].value_counts()

pie_skin_type = pye.Pie('성별 피부타입 비율 [좌: 여성, 우: 남성]',title_pos='center')
pie_skin_type.add('여성',f_skin_type_cnt.index, f_skin_type_cnt,
            center=[30,50],
            radius=[20,50],
           is_label_show=True)
pie_skin_type.add('남성',m_skin_type_cnt.index, m_skin_type_cnt,
            center=[70,50],
            radius=[20,50],
           is_label_show=True,
           legend_orient='vertical',
           legend_pos='left')
pie_skin_type

In [37]:
pie_skin_type.render('../images/graphs/sex_skin_type.html')

# 성별 선호도 비율

In [38]:
f_rate_cnt = glowpick.loc[glowpick.sex=='Female','rate'].value_counts()
m_rate_cnt = glowpick.loc[glowpick.sex=='Male','rate'].value_counts()

pie_rate = pye.Pie('성별 선호도 비율 [좌: 여성, 우: 남성]',title_pos='center')
pie_rate.add('여성',f_rate_cnt.index, f_rate_cnt,
            center=[30,50],
            radius=[20,50],
           is_label_show=True)
pie_rate.add('남성',m_rate_cnt.index, m_rate_cnt,
            center=[70,50],
            radius=[20,50],
           is_label_show=True,
           legend_orient='vertical',
           legend_pos='left')
pie_rate

In [39]:
pie_rate.render('../images/graphs/sex_rate.html')

# 성별 평점 비율
큰 차이 없어서 안써도될듯

In [40]:
star_lst = []
star_lst.append(glowpick.loc[glowpick.sex=='Female', 'star'].tolist())
star_lst.append(glowpick.loc[glowpick.sex=='Male', 'star'].tolist())

In [41]:
boxplot_star = pye.Boxplot("성별 평점 비율", title_pos='center')
x_axis = ['Female','Male']
y_axis = boxplot_star.prepare_data(star_lst)
boxplot_star.add("", x_axis, y_axis)
boxplot_star

# 성별에 따라 카테고리 빈도

남자가 여자보다 다른 카테고리에 비해 메이크업에 리뷰를 많이 남기는 편임

In [42]:
sex_cat = glowpick.groupby('sex').category.value_counts(normalize=True).unstack().T
sex_cat = round(sex_cat,3)
sex_cat = sex_cat.sort_values('Male',ascending=False)
bar_cat = pye.Bar('성별에 따라 카테고리 빈도', width=1000)
bar_cat.add('여자',sex_cat.index, sex_cat['Female'], xaxis_rotate=30, mark_line=['max'])
bar_cat.add('남자',sex_cat.index, sex_cat['Male'], xaxis_rotate=30, mark_line=['max'])
bar_cat

In [60]:
glowpick

,date,user_id,sex,age_skin_type,rate,content,product_url,brand,product,vol_price,star,nb_reviews,sales_rank,category,unique_id,age,skin_type
0,2개월 전,현콕,Male,28세 · 복합성 ·,good,제일 무난하고 가볍게쓰기 좋아서 몇년째 이거만써요,/product/7212,이니스프리(innisfree),포레스트 포맨 모이스처 스킨,"180ml19,000원",4.07,(81),1,스킨,0,30대,복합성
1,2개월 전,sso___,Female,28세 · 복합성 ·,soso,남자친구 선물로 사줬는데 그저그렇데요 나쁘지도 좋지도 않데요,/product/7212,이니스프리(innisfree),포레스트 포맨 모이스처 스킨,"180ml19,000원",4.07,(81),1,스킨,1,30대,복합성
2,3개월 전,minseo6666,Female,17세 · 복합성 ·,good,아빠 사드렸는데 저아해여,/product/7212,이니스프리(innisfree),포레스트 포맨 모이스처 스킨,"180ml19,000원",4.07,(81),1,스킨,2,20대,복합성
3,4개월 전,피부에관심있는남자,Male,21세 · 복합성 ·,good,향이 좋아요 근데 좀 제 피부가 민감해서 안맞는거 같아요 좀 올라오더라고요ㅠ,/product/7212,이니스프리(innisfree),포레스트 포맨 모이스처 스킨,"180ml19,000원",4.07,(81),1,스킨,3,30대,복합성
4,5개월 전,타라타라,Female,24세 · 복합성 ·,good,가격이 저렴하고 냄새가 안 쎄서 좋아요. 👍 다음에도 또 구매할 것 같아요.,/product/7212,이니스프리(innisfree),포레스트 포맨 모이스처 스킨,"180ml19,000원",4.07,(81),1,스킨,4,30대,복합성
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5796,2015.3.13,블링족제비,Male,26세 · 건성 ·,worst,그냥 딱풀을 사다가 옆머리 굳히는데 쓰는게 더 좋지않을까,/product/2606,미쟝센,머그샷 파파라치 글루왁스,"100g13,000원",2.25,(8),73,헤어스타일링,5796,30대,건성
5797,2014.10.1,YJEY,Female,20세 · 지성 ·,worst,잘 나오지도 않고 잘 되지도 않고.. 사촌오빠 줬다가 불편 폭탄 들음ㅠㅠㅡㅡ,/product/2606,미쟝센,머그샷 파파라치 글루왁스,"100g13,000원",2.25,(8),73,헤어스타일링,5797,30대,지성
5798,2014.9.23,tvirus,Male,41세 · 복합성 ·,worst,왁스자체를 떠나서 구멍이 매일 막혀서 짜도 나오질 않아요. 통자체가 불량인듯!!,/product/2606,미쟝센,머그샷 파파라치 글루왁스,"100g13,000원",2.25,(8),73,헤어스타일링,5798,50대,복합성
5799,2018.6.13,그런데,Female,16세 · 지성 ·,soso,제가 마블 말고 오리지널을 샀는데 그게 없어서 여기다가 쓸게요. 우선은 좋아요. 좋...,/product/94695,더페이스샵,[단종] 마블 픽스온 워터 스프레이,"200ml9,000원",3.00,(1),-,헤어스타일링,5799,20대,지성


# 성별 사용 화장품의 가격 / 카테고리별 가격

TODO: vol_price 나눠야함

In [33]:
for s in ['ea','ml','g']:
    glowpick['vol_price'] = glowpick.vol_price.str.replace(s,' ')

In [34]:
glowpick.vol_price.str.split(' ',expand=True).iloc[:,2].unique()

array([None, '14,000원', '15,000원', '4,900원', '52,000원', '9,000원'],
      dtype=object)

In [35]:
glowpick[glowpick.vol_price=='-36,000원']

,date,user_id,sex,age_skin_type,rate,content,product_url,brand,product,vol_price,star,nb_reviews,sales_rank,category,unique_id,age,skin_type
4591,2018.11.10,superstars,Male,34세 · 복합성 ·,good,"장점 : 발림성, 지속력?",/product/85004,레오 티지아노,뚜또베네 젠틀 선 비비 로션,"-36,000원",4.0,(1),25,선케어,4591,40대,복합성


In [212]:
glowpick.groupby('sex').category.value_counts()

sex     category
Female  로션          665
        스킨          575
        에센스         534
        헤어스타일링      384
        선케어         278
        메이크업        240
        클렌징         225
        헤어/바디       221
        크림          204
        쉐이빙         127
        애프터쉐이브       99
        스크럽/필링       22
        마스크/팩        15
Male    메이크업        300
        에센스         270
        로션          265
        스킨          259
        헤어스타일링      222
        쉐이빙         171
        선케어         161
        클렌징         148
        크림          120
        헤어/바디       112
        애프터쉐이브       41
        마스크/팩        17
        스크럽/필링       14
Name: category, dtype: int64

# 성별 사용 화장품 용량

# 성별 리뷰를 남기는 브랜드/상품/카테고리/유형